In [1]:
import os 

In [2]:
%pwd

'/Users/sarthaksonpatki/Desktop/My Files/Text Summarizer/Text_Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sarthaksonpatki/Desktop/My Files/Text Summarizer/Text_Summarizer'

In [12]:
from  dataclasses import dataclass
from pathlib import Path


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [13]:
from text_Summarizer.constants import * 
from text_Summarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_flepath= CONFIG_FILE_PATH,
        params_pfileath= PARAMS_FILE_PATH,):

        self.config = read_yaml(config_flepath)
        self.params = read_yaml(params_pfileath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from text_Summarizer.logging import logger
from text_Summarizer.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-09 18:58:42,392: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-09 18:58:42,394: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-09 18:58:42,395: INFO: common: created directory at: artifacts]
[2024-10-09 18:58:42,396: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-09 18:58:43,189: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 83EE:3B55E4:1F5B65:21E2A8:670718B2
Accept-Ranges: bytes
Date: Wed, 09 Oct 2024 23:58:43 GMT
Via: 1.1 varnish
X-Served-By: cache-chi-kigq8000114-CHI
X-C